In [25]:
#Import Modules
import pandas as pd 
import matplotlib.pyplot as plt
import panel as pn
from panel.interact import interact
import plotly.express as px
pn.extension("plotly")
import hvplot.pandas
%matplotlib inline

In [26]:
import import_ipynb


In [27]:
# Define function for analyzing Unemployment Insurance Claims 
def insurance_claims():
    unemployment_insurance_data=pd.read_csv('resources/Unemployment Insurance.csv',parse_dates=True,infer_datetime_format=True,index_col='DATE')
    unempl_claims_last2yrs = unemployment_insurance_data.loc[unemployment_insurance_data.index > '2018-01-01']
    return unempl_claims_last2yrs.hvplot(kind='line', 
                                         title='Unemployment Insurance Claims (2018-2020)',
                                         rot=90,
                                         xlabel = 'Date',
                                         ylabel = 'Insurance Claims',
                                         height = 400
                                        )

In [28]:
# Define function for analyzing change in Unemployment Rate
def unemployment_rate():
    unemployment_rate=pd.read_csv('resources/Unemployment Rate.csv',parse_dates=True,infer_datetime_format=True,index_col='DATE')
    unempl_rate_last2yrs = unemployment_rate.loc[unemployment_rate.index > '2018-01-01']
    unempl_rate_last2yrs = unempl_rate_last2yrs.rename(columns={'UNRATE':'unempl_rate'})
    unempl_rate_plot = unempl_rate_last2yrs.hvplot(kind='bar',
                                        rot=90,
                                        title='Unemployment Rate (2018-2020)',
                                        xlabel = 'Date',
                                        ylabel = 'Unemployment Rate',
                                        height = 400
                                       )
    
    return unempl_rate_plot

In [30]:
# Define function for analyzing COVID Data
def covid_data(string):
    all_covid_data=pd.read_csv('resources/covid_data.csv')
    usa_covid_data=all_covid_data.loc[all_covid_data['iso_code'] == 'USA']
    usa_covid_data=usa_covid_data.rename(columns={"date":"DATE"})
    usa_covid_data=usa_covid_data.dropna()
    usa_covid_data=usa_covid_data.set_index('DATE')
    usa_covid_data.index = pd.to_datetime(usa_covid_data.index)

    usa_covid_stats = usa_covid_data.filter(items=['total_cases','total_deaths'])
    usa_covid_stats_by_month = usa_covid_stats.groupby([pd.Grouper(freq='M')])['total_cases','total_deaths'].sum()
    
    if string == 'Daily':
        return usa_covid_stats
    elif string == 'Monthly':
        return usa_covid_stats_by_month

In [31]:
# Define function for analyzing Mobility Data
def mobility_data():
    mobility_data = pd.read_csv('resources/2020_US_Region_Mobility_Report.csv',index_col='date',infer_datetime_format=True,parse_dates=True)
    mobility_data = mobility_data.filter(['date','retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline','residential_percent_change_from_baseline'])
    mobility_data.rename(columns={'retail_and_recreation_percent_change_from_baseline':'Retail_Recreation','grocery_and_pharmacy_percent_change_from_baseline':'Grocery_Pharmacy','parks_percent_change_from_baseline':'Parks','transit_stations_percent_change_from_baseline':'Public_Transit','workplaces_percent_change_from_baseline':'Workplaces','residential_percent_change_from_baseline':'Residential'},inplace=True)
    mobility_data.dropna(inplace=True)
    mobility_data_by_month = mobility_data.groupby([pd.Grouper(freq='M')])['Retail_Recreation','Grocery_Pharmacy','Parks','Public_Transit','Workplaces','Residential'].sum()
    return mobility_data_by_month

In [32]:
# Define function for parallel coordinates
def covid_mobility_parallel():
    usa_covid_stats_by_month = covid_data('Monthly')
    mobility_data_by_month = mobility_data()
    covid_mobility_grp = pd.concat([usa_covid_stats_by_month,mobility_data_by_month],join='inner',axis=1)
    covid_mobility_grp.reset_index()
    
    return px.parallel_coordinates(
        covid_mobility_grp,
        dimensions = ['total_cases','Retail_Recreation','Grocery_Pharmacy','Parks','Public_Transit','Workplaces'],
        color = 'total_cases',
        color_continuous_scale = px.colors.sequential.Inferno,
        height = 700
    )
       

In [33]:
# Create panel for Unemployment visualizations
unemployment = pn.Row(unemployment_rate(),insurance_claims())

# Create panel for COVID & Mobility Visualizations
covid_data_plot = covid_data('Daily').hvplot.scatter(y=['total_cases'],
                                                     rot=90,
                                                     #xlim=(pd.to_datetime('2020-02-15'),pd.to_datetime('2020-10-31')),
                                                     xlabel = 'Date',
                                                     ylabel = 'No. of Cases',
                                                     height = 400,
                                                     width = 1200,
                                                     title = 'USA Total Cases 2020'
                                                    )

mobility_by_month_plot = mobility_data().hvplot.bar(x='date',
                                  y=('Retail_Recreation','Grocery_Pharmacy','Parks','Public_Transit','Workplaces'),
                                  rot=90,
                                  xlabel = 'Date',
                                  ylabel = 'Per Cent Change',
                                  height = 600,
                                  width=1200,
                                  title='Per Cent Change in Mobility')
covid_mobility = pn.Column(covid_data_plot,mobility_by_month_plot)

# Create panel for Parallel Coordinates
cov_mob_parallel = pn.panel(covid_mobility_parallel())


In [34]:
#youssef's code 

import FinancialAnaylsis

In [35]:
## Carla's Code


In [36]:
dashboard = pn.Tabs(
    ('Impact on Mobility', covid_mobility),
    ('Impact on Jobs', unemployment),
    ('Correlation Pararell', covid_mobility_parallel),
    ('Stock and Fundementals', pn.Column(stock_and_fundemental1 ,stock_and_fundemental2,stock_and_fundemental3,stock_and_fundemental4, stock_and_fundemental5 )), 
    ('Mobility Impact on Company Fundementals', pn.Column(mobility_and_fundemental1, mobility_and_fundemental2,mobility_and_fundemental3, mobility_and_fundemental4, mobility_and_fundemental5))
)

In [38]:
dashboard.servable()

Tabs
    [0] Column
        [0] HoloViews(Scatter)
        [1] HoloViews(Bars)
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
    [2] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive12776')
    [3] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12644')
                    [0] HoloViews(Layout, name='interactive12644')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12657')
                    [0] HoloViews(Layout, name='interactive12657')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12670')
                    [0] HoloViews(Layout, name='interactive12670')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [3] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12683')
                    [0] HoloViews(Layout, name='interactive12683')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [4] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12696')
                    [0] HoloViews(Layout, name='interactive12696')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
    [4] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12710')
                    [0] HoloViews(Layout, name='interactive12710')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12723')
                    [0] HoloViews(Layout, name='interactive12723')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12736')
                    [0] HoloViews(Layout, name='interactive12736')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', options=['AMZN', 'ETSY', ...], value='AMZN', width=250)
                        [1] VSpacer()
        [3] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive12749')
                    [0] HoloViews(Layout, name='interactive12749')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='Company', o